<a href="https://colab.research.google.com/github/liangnic/Kaggles/blob/main/Retail%20Sales%20Analysis/Sales_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [8]:
# Packages Import
import pandas as pd
import numpy as np

# Files Import
INVENTORY_CSV = pd.read_csv('/content/vend-total_revenue-for-inventory_product-by-all (2025-08-04 to 2025-08-25).csv')
WEEKLY_CSV = pd.read_csv('/content/vend-total_revenue-sales_summary-by-week (2025-08-04 to 2025-08-26).csv')

# Step 0. Data Preparation

## Cleaning

* **Cleaning:** Remove currency/percentage symbols, convert to numeric types.
* **Uniqueness check:** Verify whether SKU corresponds one-to-one with Product.
* **Derived fields:**

    * **Average Selling Price** = Revenue / Items Sold
    * **Product Gross Margin** = Gross Profit / Revenue
    * **Inventory-to-Revenue Ratio** = Inventory Cost / Revenue

In [9]:
INVENTORY_CSV.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5913 entries, 0 to 5912
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Product                   5913 non-null   object 
 1   SKU                       5913 non-null   object 
 2   Supplier Code             5911 non-null   object 
 3   Brand                     5911 non-null   object 
 4   Supplier                  5908 non-null   object 
 5   Category                  5894 non-null   object 
 6   Tag                       5815 non-null   object 
 7   Closing Inventory         5913 non-null   float64
 8   Items Sold per Day        5913 non-null   float64
 9   Items Sold                5913 non-null   float64
 10  Days Cover                5913 non-null   float64
 11  Sell-through Rate         5913 non-null   object 
 12  Revenue                   5913 non-null   object 
 13  Gross Profit              5913 non-null   object 
 14  Margin (

In [10]:
def clean_dataset(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean the retail dataset by:
      1. Dropping 'Supplier Code' and 'Supplier' columns
      2. Splitting the 'Brand' column into two columns: 'Country' and 'Brand'
      3. Removing currency/percentage symbols and converting to numeric types

    Parameters
    ----------
    df : pd.DataFrame
        Input dataset.

    Returns
    -------
    pd.DataFrame
        A cleaned DataFrame with standardized numeric types and separated columns.
    """

    # Drop supplier-related columns
    df = df.drop(columns=["Supplier Code", "Supplier"], errors="ignore")

    # Split 'Brand' column into 'Country' and 'Brand'
    df[["Country", "Brand"]] = df["Brand"].str.split(" ", n=1, expand=True)

    # Columns to process
    currency_cols = ["Revenue", "Gross Profit", "Inventory Cost", "Retail Value (Excl. Tax)"]
    percent_cols = ["Sell-through Rate", "Margin (%)"]

    # Remove $ and , then convert to float
    for col in currency_cols:
        if col in df.columns:
            df[col] = df[col].astype(str).str.replace("[$,]", "", regex=True).astype(float)

    # Remove % and convert to float
    for col in percent_cols:
        if col in df.columns:
            df[col] = df[col].astype(str).str.replace("%", "", regex=True).astype(float)
            # Uncomment if you prefer 0-1 scale instead of 0-100
            # df[col] = df[col] / 100.0

    return df

In [11]:
INVENTORY_CSV = clean_dataset(INVENTORY_CSV)

In [13]:
INVENTORY_CSV.head(1)

,Product,SKU,Brand,Category,Tag,Closing Inventory,Items Sold per Day,Items Sold,Days Cover,Sell-through Rate,Revenue,Gross Profit,Margin (%),Inventory Cost,Retail Value (Excl. Tax),Country
0,Tirtir Mask Fit Red Cushion,8809928133867,Tirtir,Makeup / Face Makeup / Makeup Cushion,Cushion,146.0,4.3,1948.0,34.0,93.03,61849.31,24837.31,40.16,2774.0,5043.64,KR
